In [1]:
"""
Topic: Basics of AWS Lambda (Event-Driven Jobs)
===============================================
Simulates how AWS Lambda functions automatically trigger 
on S3 file uploads and process incoming data events.
"""

import json
import boto3
import logging

# -------------------------------------------------------------------
# Step 1 – Setup Logging
# -------------------------------------------------------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
logger = logging.getLogger()

# -------------------------------------------------------------------
# Step 2 – Simulated S3 Event
# -------------------------------------------------------------------
def generate_sample_event(bucket_name, key_name):
    """Simulate an S3 event notification"""
    return {
        "Records": [
            {
                "s3": {
                    "bucket": {"name": bucket_name},
                    "object": {"key": key_name}
                }
            }
        ]
    }

# -------------------------------------------------------------------
# Step 3 – Lambda Handler (Main Logic)
# -------------------------------------------------------------------
def lambda_handler(event, context=None):
    """Main Lambda entry point"""
    logger.info("📥 Event received.")
    try:
        record = event["Records"][0]
        bucket = record["s3"]["bucket"]["name"]
        key = record["s3"]["object"]["key"]

        logger.info(f"Triggered by file: {key} in bucket: {bucket}")

        # Example processing logic
        if key.endswith(".json"):
            logger.info("Processing JSON file and moving to trusted zone...")
            s3 = boto3.client("s3")
            source = {"Bucket": bucket, "Key": key}
            destination_key = key.replace("raw/", "trusted/")
            s3.copy_object(Bucket=bucket, CopySource=source, Key=destination_key)
            logger.info(f"✅ File moved to {destination_key}")
        else:
            logger.warning("Skipping non-JSON file.")

        return {"statusCode": 200, "body": "Processing complete"}

    except Exception as e:
        logger.error(f"❌ Error processing event: {e}")
        return {"statusCode": 500, "body": str(e)}

# -------------------------------------------------------------------
# Step 4 – Local Simulation
# -------------------------------------------------------------------
if __name__ == "__main__":
    # Simulate S3 event trigger
    test_event = generate_sample_event(
        "supplychain-data-demo",
        "raw/sample_shipment.json"
    )
    response = lambda_handler(test_event)
    print("\nLambda Execution Response:", json.dumps(response, indent=2))


2025-11-10 15:35:28,035 - 📥 Event received.
2025-11-10 15:35:28,043 - Triggered by file: raw/sample_shipment.json in bucket: supplychain-data-demo
2025-11-10 15:35:28,043 - Processing JSON file and moving to trusted zone...
2025-11-10 15:35:30,887 - ❌ Error processing event: Unable to locate credentials



Lambda Execution Response: {
  "statusCode": 500,
  "body": "Unable to locate credentials"
}
